In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression as LR, Ridge
from sklearn.ensemble import RandomForestRegressor as RF
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error as mse, r2_score as r2
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [2]:
data_train=pd.read_csv('train.csv')

data_test=pd.read_csv('test.csv')

### общие преобразования, явные опечатки

In [3]:
#тренировочный
#слишком много комнат
data_train.loc[data_train['Rooms']==10,'Rooms']=2
data_train.loc[data_train['Rooms']==19,'Rooms']=1
data_train.loc[data_train['Rooms']==6,'Rooms']=2
data_train.loc[(data_train['Rooms']>3) & (data_train['Square']<61),'Rooms']=1

#комнат 0
data_train.loc[[2269,6149,4853],'Rooms']=1
data_train.loc[[3911],'Rooms']=2
data_train.loc[[4366],'Rooms']=3
data_train.loc[[8834],'Rooms']=4
data_train.loc[[1397,1981],'Rooms']=5

#возможные варианты комнатности
rooms_train = data_train['Rooms'].unique()
rooms_train.sort(axis=0)

#средняя площадь по комнатности
square_room_train = []
for i in rooms_train:
    square_room_train.append(data_train.loc[data_train['Rooms']==i,'Square'].mean())

#проблемный столбец лучше убрать, не понятно чем заполнять - средним тоже не правильно
data_train.drop('Healthcare_1',axis=1,inplace=True)

#неверно год дома
data_train.loc[data_train['HouseYear']==20052011,'HouseYear']=2008
data_train.loc[data_train['HouseYear']==4968,'HouseYear']=1968

#записи с пустыми LifeSquare заливаем средним между квартирами с идентичной комнатностью
square_lifesquare_train = []
for i in rooms_train:
    square_lifesquare_train.append((data_train.loc[data_train['Rooms']==i,'Square']/data_train.loc[data_train['Rooms']==i,'LifeSquare']).mean())
    data_train.loc[(data_train['Rooms']==i)&(data_train['LifeSquare'].isnull()),'LifeSquare']=data_train['Square']/square_lifesquare_train[int(i-1)]

#0-этажный дом
data_train.loc[data_train['HouseFloor'] == 0, 'HouseFloor'] = data_train['Floor']

#этаж выше этажей в доме
#поменять местами
index_array_floor=data_train.loc[data_train['Floor'] > data_train['HouseFloor']].index
data_train.loc[index_array_floor,'Floor'] = data_train['Floor'] + data_train['HouseFloor']
data_train.loc[index_array_floor,'HouseFloor'] = data_train['Floor'] - data_train['HouseFloor']
data_train.loc[index_array_floor,'Floor'] = data_train['Floor'] - data_train['HouseFloor']

In [4]:
#тренировочный
#общая маленькая < 11
data_train.loc[(data_train['Square']<11)&(data_train['LifeSquare']<11),'LifeSquare'] = data_train['LifeSquare']*10
data_train.loc[data_train['Square']<11,'Square'] = data_train['Square']*10

#общая большая > 300
data_train.loc[[4262,4690,6977],'Square'] = data_train['Square']/10
data_train.loc[[4262,4690,6977],'LifeSquare'] = data_train['LifeSquare']/10

#жилая маленькая < 11
data_train.loc[data_train['LifeSquare']<1,'LifeSquare'] = data_train['LifeSquare']*100
data_train.loc[data_train['LifeSquare']<11,'LifeSquare'] = data_train['LifeSquare']*10

#жилая большая > 300
data_train.loc[data_train['LifeSquare']>2000,'LifeSquare'] = data_train['LifeSquare']/100
data_train.loc[data_train['LifeSquare']>200,'LifeSquare'] = data_train['LifeSquare']/10

#жилая больше общей
index_array_square=data_train.loc[data_train['LifeSquare']>(data_train['Square'])].index

#поменять местами
data_train.loc[index_array_square,'Square'] = data_train['Square'] + data_train['LifeSquare']
data_train.loc[index_array_square,'LifeSquare'] = data_train['Square'] - data_train['LifeSquare']
data_train.loc[index_array_square,'Square'] = data_train['Square'] - data_train['LifeSquare']

In [5]:
#тренировочный
#жилая в неверной пропорции от общей
for i in rooms_train:
    data_train.loc[(data_train['Rooms']==i)&(data_train['LifeSquare']<(data_train['Square']/square_lifesquare_train[int(i-1)])),'LifeSquare'] = data_train['Square']/square_lifesquare_train[int(i-1)]
    data_train.loc[(data_train['Rooms']==i)&(data_train['LifeSquare']>(data_train['Square']/square_lifesquare_train[int(i-1)])),'LifeSquare'] = data_train['Square']/square_lifesquare_train[int(i-1)]

In [6]:
#тестовый
#слишком много комнат
data_test.loc[data_test['Rooms']==17,'Rooms']=2
data_test.loc[(data_test['Rooms']==4) & (data_test['Square']<61),'Rooms']=1
data_test.loc[(data_test['Rooms']==5) & (data_test['Square']<61),'Rooms']=2
data_test.loc[data_test['Rooms']==6,'Rooms']=5

#комнат 0
data_test.loc[(data_test['Rooms']==0) & (data_test['Square']>80),'Rooms']=5
data_test.loc[(data_test['Rooms']==0) & (data_test['Square']<80),'Rooms']=3

#возможные варианты комнатности
rooms_test = data_test['Rooms'].unique()
rooms_test.sort(axis=0)

#средняя площадь по комнатности
square_room_test = []
for i in rooms_test:
    square_room_test.append(data_test.loc[data_test['Rooms']==i,'Square'].mean())

#проблемный столбец лучше убрать, не понятно чем заполнять - средним тоже не правильно
data_test.drop('Healthcare_1',axis=1,inplace=True)

#записи с пустыми LifeSquare заливаем средним между квартирами с идентичной комнатностью
square_lifesquare_test = []
for i in rooms_test:
    square_lifesquare_test.append((data_test.loc[data_test['Rooms']==i,'Square']/data_test.loc[data_test['Rooms']==i,'LifeSquare']).mean())
    data_test.loc[(data_test['Rooms']==i)&(data_test['LifeSquare'].isnull()),'LifeSquare']=data_test['Square']/square_lifesquare_test[int(i-1)]

#0-этажный дом
data_test.loc[data_test['HouseFloor'] == 0, 'HouseFloor'] = data_test['Floor']

#этаж выше этажей в доме
#поменять местами
index_array_floor=data_test.loc[data_test['Floor'] > data_test['HouseFloor']].index
data_test.loc[index_array_floor,'Floor'] = data_test['Floor'] + data_test['HouseFloor']
data_test.loc[index_array_floor,'HouseFloor'] = data_test['Floor'] - data_test['HouseFloor']
data_test.loc[index_array_floor,'Floor'] = data_test['Floor'] - data_test['HouseFloor']

In [7]:
#тестовый
#общая маленькая < 11
data_test.loc[(data_test['Square']<11)&(data_test['LifeSquare']<11),'LifeSquare'] = data_test['LifeSquare']*10
data_test.loc[data_test['Square']<11,'Square'] = data_test['Square']*10

#жилая маленькая < 11
data_test.loc[data_test['LifeSquare']<1,'LifeSquare'] = data_test['LifeSquare']*100
data_test.loc[data_test['LifeSquare']<11,'LifeSquare'] = data_test['LifeSquare']*10

#жилая большая > 300
data_test.loc[data_test['LifeSquare']>200,'LifeSquare'] = data_test['LifeSquare']/10

#жилая больше общей
index_array_square=data_test.loc[data_test['LifeSquare']>(data_test['Square'])].index
#поменять местами
data_test.loc[index_array_square,'Square'] = data_test['Square'] + data_test['LifeSquare']
data_test.loc[index_array_square,'LifeSquare'] = data_test['Square'] - data_test['LifeSquare']
data_test.loc[index_array_square,'Square'] = data_test['Square'] - data_test['LifeSquare']

In [8]:
#тестовый
#жилая в неверной пропорции от общей
for i in rooms_test:
    data_test.loc[(data_test['Rooms']==i)&(data_test['LifeSquare']<(data_test['Square']/square_lifesquare_test[int(i-1)])),'LifeSquare'] = data_test['Square']/square_lifesquare_test[int(i-1)]
    data_test.loc[(data_test['Rooms']==i)&(data_test['LifeSquare']>(data_test['Square']/square_lifesquare_test[int(i-1)])),'LifeSquare'] = data_test['Square']/square_lifesquare_test[int(i-1)]

### прогноз комнатности

In [9]:
#тренировочный
stats=data_train.groupby(['Rooms'],as_index=False)['Square'].mean().rename(columns={'Square':'mean_square_room'})
data_train=pd.merge(data_train,stats,on=['Rooms'],how='left')
data_train['square_room_diff'] = abs(data_train['Square'] - data_train['mean_square_room'])

In [10]:
#тренировочный
y = data_train['Rooms']
X = data_train.loc[:,['Square','mean_square_room']]

X_train = data_train.loc[data_train['square_room_diff'] < 5,['Square','mean_square_room']]
X_valid = data_train.loc[data_train['square_room_diff'] >= 5,['Square','mean_square_room']]
y_train = data_train.loc[data_train['square_room_diff'] < 5,['Rooms']]
y_valid = data_train.loc[data_train['square_room_diff'] >= 5,['Rooms']]

lr = DecisionTreeClassifier()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_valid)
y_pred1 = lr.predict(data_train.loc[:,['Square','mean_square_room']])
data_train['pred_room'] = y_pred1

In [11]:
#тестовый
stats=data_test.groupby(['Rooms'],as_index=False)['Square'].mean().rename(columns={'Square':'mean_square_room'})
data_test=pd.merge(data_test,stats,on=['Rooms'],how='left')
data_test['square_room_diff'] = abs(data_test['Square'] - data_test['mean_square_room'])

In [12]:
#тестовый
y = data_test['Rooms']
X = data_test.loc[:,['Square','mean_square_room']]

X_train = data_test.loc[data_test['square_room_diff'] < 5,['Square','mean_square_room']]
X_valid = data_test.loc[data_test['square_room_diff'] >= 5,['Square','mean_square_room']]
y_train = data_test.loc[data_test['square_room_diff'] < 5,['Rooms']]
y_valid = data_test.loc[data_test['square_room_diff'] >= 5,['Rooms']]

lr = DecisionTreeClassifier()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_valid)
y_pred1 = lr.predict(data_test.loc[:,['Square','mean_square_room']])
data_test['pred_room'] = y_pred1

### прогноз жилой от комнатности и общей площади

In [13]:
#тренировочный
features=data_train[['Rooms','Square']]
target=data_train['LifeSquare']
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.3)
model1=RF(n_estimators=100,max_depth=10,min_samples_leaf=5)
model1.fit(X_train,y_train)
y_pred=model1.predict(X_test)
y_pred1 = model1.predict(data_train.loc[:,['Rooms','Square']])
data_train['pred_life_square'] = y_pred1
print(mse(data_train['LifeSquare'],y_pred1),r2(data_train['LifeSquare'],y_pred1))
data_train['life_square_diff'] = abs(data_train['LifeSquare'] - data_train['pred_life_square'])

1.8699547750230925 0.9810469922605816


In [14]:
#тестовый
features=data_test[['Rooms','Square']]
target=data_test['LifeSquare']
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.3)
model1=RF(n_estimators=100,max_depth=10,min_samples_leaf=5)
model1.fit(X_train,y_train)
y_pred=model1.predict(X_test)
y_pred1 = model1.predict(data_test.loc[:,['Rooms','Square']])
data_test['pred_life_square'] = y_pred1
print(mse(data_test['LifeSquare'],y_pred1),r2(data_test['LifeSquare'],y_pred1))
data_test['life_square_diff'] = abs(data_test['LifeSquare'] - data_test['pred_life_square'])

1.4777257666452577 0.9769480824999172


### группировка по району и комнатности

In [15]:
#тренировочный
distr_info=data_train['DistrictId'].value_counts().reset_index().rename(columns={'DistrictId':'house_count_district','index':'DistrictId'})

distr_info['district_is_large']=0
distr_info.loc[distr_info['house_count_district']<10, 'district_is_large'] = 0
distr_info.loc[(distr_info['house_count_district']>=10)&(distr_info['house_count_district']<30), 'district_is_large'] = 1
distr_info.loc[(distr_info['house_count_district']>=30)&(distr_info['house_count_district']<50), 'district_is_large'] = 2
distr_info.loc[(distr_info['house_count_district']>=50)&(distr_info['house_count_district']<100), 'district_is_large'] = 3
distr_info.loc[distr_info['house_count_district']>=100, 'district_is_large'] = 4

data_train=pd.merge(data_train,distr_info,on='DistrictId',how='left')
#distr_info['district_is_large'].value_counts()

In [16]:
#тренировочный
stats=data_train.groupby(['district_is_large','Rooms'],as_index=False)['Price'].mean().rename(columns={'Price':'mean_price'})
data_train=pd.merge(data_train,stats,on=['district_is_large','Rooms'],how='left')

In [17]:
#тестовый
distr_info=data_test['DistrictId'].value_counts().reset_index().rename(columns={'DistrictId':'house_count_district','index':'DistrictId'})

distr_info['district_is_large']=0
distr_info.loc[distr_info['house_count_district']<10, 'district_is_large'] = 0
distr_info.loc[(distr_info['house_count_district']>=10)&(distr_info['house_count_district']<30), 'district_is_large'] = 1
distr_info.loc[(distr_info['house_count_district']>=30)&(distr_info['house_count_district']<50), 'district_is_large'] = 2
distr_info.loc[(distr_info['house_count_district']>=50)&(distr_info['house_count_district']<100), 'district_is_large'] = 3
distr_info.loc[distr_info['house_count_district']>=100, 'district_is_large'] = 4

data_test=pd.merge(data_test,distr_info,on='DistrictId',how='left')

In [18]:
#тестовый
data_test=pd.merge(data_test,stats,on=['district_is_large','Rooms'],how='left')

### группировка по году домов

In [19]:
#тренировочный
data_train['old_house']=0
data_train.loc[data_train['HouseYear']<1960, 'old_house'] = 0
data_train.loc[(data_train['HouseYear']>=1960)&(data_train['HouseYear']<1970), 'old_house'] = 1 
data_train.loc[(data_train['HouseYear']>=1970)&(data_train['HouseYear']<1980), 'old_house'] = 2 
data_train.loc[(data_train['HouseYear']>=1980)&(data_train['HouseYear']<1990), 'old_house'] = 3 
data_train.loc[(data_train['HouseYear']>=1990)&(data_train['HouseYear']<2000), 'old_house'] = 4 
data_train.loc[(data_train['HouseYear']>=2000)&(data_train['HouseYear']<2010), 'old_house'] = 5 
data_train.loc[data_train['HouseYear']>=2010, 'old_house'] = 6

In [20]:
#тренировочный
stats=data_train.groupby(['old_house','Rooms'],as_index=False)['Price'].mean().rename(columns={'Price':'mean_price_old_house_room'})
data_train=pd.merge(data_train,stats,on=['old_house','Rooms'],how='left')

In [21]:
#тестовый
data_test['old_house']=0
data_test.loc[data_test['HouseYear']<1960, 'old_house'] = 0
data_test.loc[(data_test['HouseYear']>=1960)&(data_test['HouseYear']<1970), 'old_house'] = 1 
data_test.loc[(data_test['HouseYear']>=1970)&(data_test['HouseYear']<1980), 'old_house'] = 2 
data_test.loc[(data_test['HouseYear']>=1980)&(data_test['HouseYear']<1990), 'old_house'] = 3 
data_test.loc[(data_test['HouseYear']>=1990)&(data_test['HouseYear']<2000), 'old_house'] = 4 
data_test.loc[(data_test['HouseYear']>=2000)&(data_test['HouseYear']<2010), 'old_house'] = 5 
data_test.loc[data_test['HouseYear']>=2010, 'old_house'] = 6

In [22]:
#тестовый
data_test=pd.merge(data_test,stats,on=['old_house','Rooms'],how='left')

### группировка по площади и району

In [23]:
#тренировочный
data_train['square_group'] = 0
data_train.loc[data_train['Square']<45,'square_group'] = 0
data_train.loc[(data_train['Square']>=40)&(data_train['Square']<60),'square_group'] = 1
data_train.loc[(data_train['Square']>=60)&(data_train['Square']<80),'square_group'] = 2
data_train.loc[(data_train['Square']>=80)&(data_train['Square']<100),'square_group'] = 3
data_train.loc[data_train['Square']>=100,'square_group'] = 4

In [24]:
#тренировочный
stats=data_train.groupby(['district_is_large','square_group'],as_index=False)['Price'].mean().rename(columns={'Price':'mean_price_square_group'})
data_train=pd.merge(data_train,stats,on=['district_is_large','square_group'],how='left')

In [25]:
#тестовый
data_test['square_group'] = 0
data_test.loc[data_test['Square']<45,'square_group'] = 0
data_test.loc[(data_test['Square']>=40)&(data_test['Square']<60),'square_group'] = 1
data_test.loc[(data_test['Square']>=60)&(data_test['Square']<80),'square_group'] = 2
data_test.loc[(data_test['Square']>=80)&(data_test['Square']<100),'square_group'] = 3
data_test.loc[data_test['Square']>=100,'square_group'] = 4

In [26]:
#тестовый
data_test=pd.merge(data_test,stats,on=['district_is_large','square_group'],how='left')

## МОДЕЛИ

In [78]:
features=data_train[['DistrictId','pred_room','pred_life_square','Square','Floor','HouseFloor','HouseYear','mean_price','mean_price_old_house_room']]
target=data_train['Price']

In [79]:
model3=RF(n_estimators=150,max_depth=100,min_samples_leaf=5)
X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.2,random_state=5)

In [80]:
%%time
model3.fit(X_train,y_train)

Wall time: 2.97 s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=5, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [81]:
y_pred_3=model3.predict(X_test)

In [82]:
print(mse(y_test,y_pred_3),r2(y_test,y_pred_3))

2558180250.2383676 0.6813213650587021


In [83]:
model3.feature_importances_

array([0.11344591, 0.00176528, 0.11685771, 0.11075845, 0.02839351,
       0.04773427, 0.03997853, 0.24489703, 0.29616931])

In [84]:
data_test['Price']=model3.predict(data_test.loc[:,['DistrictId','pred_room','pred_life_square','Square','Floor','HouseFloor','HouseYear','mean_price','mean_price_old_house_room']])

In [85]:
data_test.loc[:,['Id','Price']].to_csv('IMarchenko_predictions.csv',index=None)

In [86]:
pd.read_csv('IMarchenko_predictions.csv')

,Id,Price
0,725,206696.836050
1,15856,250790.781152
2,5480,181848.334163
3,15664,386034.940305
4,14275,146396.634865
5,7633,199495.225472
6,13329,179126.596487
7,5502,238060.034969
8,4220,313858.144048
9,11538,208143.114142
